<a href="https://colab.research.google.com/github/markajbell/BH/blob/main/ProcessData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import pandas as pd
import json
import os
import glob

Set the Path to the Data location (imported form SharpHound)

# Set the Path
to the Data location (imported from SharpHound) and Read json files in folder and create a dictionary with the entire data.





In [ ]:
#@title Download lab files
from IPython.display import clear_output
!wget https://raw.githubusercontent.com/markajbell/BH/refs/heads/main/mylab/20250316212118_computers.json

https://github.com/markajbell/BH/tree/main/mylab


path = "/content/Mark"
data = {}
json_files = glob.glob(os.path.join(path, "*.json"))
for file in json_files:
    filename = os.path.basename(file).replace(".json", "")
    suf = filename.split("_")[-1]

    with open(file, 'r', encoding='utf-8') as f:
        content = json.load(f)

    if "data" in content:
        if suf in data:
            data[f"{suf}"].extend(content["data"])
        else:
            data[f"{suf}"] = content["data"]
    else:
        print(f"Warning: The key 'data' was not found in {file_name}.")

Show the "content"

In [ ]:
content


{'data': [{'Properties': {'domain': 'PHANTOM.CORP',
    'name': 'SOM@PHANTOM.CORP',
    'distinguishedname': 'CN=SOM,CN=WMIPOLICY,CN=SYSTEM,DC=PHANTOM,DC=CORP',
    'domainsid': 'S-1-5-21-2697957641-2271029196-387917394',
    'isaclprotected': True},
   'ChildObjects': [],
   'Aces': [{'PrincipalSID': 'S-1-5-21-2697957641-2271029196-387917394-512',
     'PrincipalType': 'Group',
     'RightName': 'Owns',
     'IsInherited': False},
    {'PrincipalSID': 'S-1-5-21-2697957641-2271029196-387917394-512',
     'PrincipalType': 'Group',
     'RightName': 'GenericAll',
     'IsInherited': False},
    {'PrincipalSID': 'S-1-5-21-2697957641-2271029196-387917394-519',
     'PrincipalType': 'Group',
     'RightName': 'GenericAll',
     'IsInherited': False}],
   'ObjectIdentifier': '23631FBB-E0CB-47D0-8B72-A3AD167EAEF6',
   'IsDeleted': False,
   'IsACLProtected': True,
   'ContainedBy': {'ObjectIdentifier': '04F9AEFD-BFC5-4B9F-887C-FFC7F72B74F0',
    'ObjectType': 'Container'}},
  {'Properties': {

In [ ]:
data.keys()

dict_keys(['users', 'computers', 'ous', 'gpos', 'groups', 'domains', 'containers'])

In [ ]:
print("Groups")
pd.DataFrame.from_dict(data["groups"]).columns


Groups


Index(['Properties', 'Members', 'Aces', 'ObjectIdentifier', 'IsDeleted',
       'IsACLProtected', 'ContainedBy'],
      dtype='object')

In [ ]:
print("Users")
pd.DataFrame.from_dict(data["users"]).columns


Users


Index(['Properties', 'AllowedToDelegate', 'PrimaryGroupSID', 'HasSIDHistory',
       'SPNTargets', 'Aces', 'ObjectIdentifier', 'IsDeleted', 'IsACLProtected',
       'ContainedBy'],
      dtype='object')

Expand "Properties" column in new columns.

In [ ]:
print("Computers")
pd.DataFrame.from_dict(data["computers"]).columns


Computers


Index(['Properties', 'PrimaryGroupSID', 'AllowedToDelegate', 'AllowedToAct',
       'HasSIDHistory', 'DumpSMSAPassword', 'Sessions', 'PrivilegedSessions',
       'RegistrySessions', 'LocalGroups', 'UserRights', 'DCRegistryData',
       'Status', 'IsDC', 'DomainSID', 'Aces', 'ObjectIdentifier', 'IsDeleted',
       'IsACLProtected', 'ContainedBy'],
      dtype='object')

In [ ]:
print("GPOs")
pd.DataFrame.from_dict(data["gpos"]).columns


GPOs


Index(['Properties', 'Aces', 'ObjectIdentifier', 'IsDeleted', 'IsACLProtected',
       'ContainedBy'],
      dtype='object')

In [ ]:
print("OUs")
pd.DataFrame.from_dict(data["ous"]).columns

OUs


Index(['GPOChanges', 'Properties', 'Links', 'ChildObjects', 'Aces',
       'ObjectIdentifier', 'IsDeleted', 'IsACLProtected', 'ContainedBy'],
      dtype='object')

Convert to Dict

In [ ]:
JSONdata = data

In [ ]:
users_init = []
result = {"users_names": {}, "groups_names": {}, "computers_names": {}, "ous_names": {}, "gpos_names": {}}
for users in JSONdata["users"]:
    result["users_names"][users['ObjectIdentifier']] = users['Properties']['name']
    properties = users.get('Properties', {})

    row = {}
    for key in user:
        if key not in ['Properties']:
            row[key] = user[key]

    for key in properties:
        row[key] = properties[key]

    users_init.append(row)

df_users_init = pd.DataFrame.from_records(users_init)

groups_init = []
for group in JSONdata["groups"]:
    result["groups_names"][user['ObjectIdentifier']] = group['Properties']['name']
    properties = group.get('Properties', {})

    row = {}
    for key in group:
        if key not in ['Properties']:
            row[key] = group[key]

    for key in properties:
        row[key] = properties[key]

    groups_init.append(row)

df_groups_init = pd.DataFrame.from_records(groups_init)

computers_init = []

for computers in JSONdata["computers"]:
    # The key was incorrect and causing the error; it should be 'ObjectIdentifier'
    result["computers_names"][computers['ObjectIdentifier']] = computers['Properties']['name']
    properties = computers.get('Properties', {})

    row = {}
    # Iterate over the keys of the current computers object
    for key in computers:
        if key not in ['Properties']:
            row[key] = computers[key]

    for key in properties:
        row[key] = properties[key]

    computers_init.append(row)

df_computers_init = pd.DataFrame.from_records(computers_init)

#ous_init = []

#for ous in JSONdata["ous"]:
#    result["ous_names"][ous['ContainedBy']['ObjectIdentifier']] = ous['Properties']['name']
#    properties = ous.get('Properties', {})###
#
#    row = {}
#    for key in ous: # This line uses 'user' which is defined outside the loop and might contain the 'AllowedToDelegate' key.
#        if key not in ['Properties']:
#            row[key] = ous[key] #This line tries to access 'AllowedToDelegate' within 'ous' which may not exist.#
#
#    for key in properties:
#        row[key] = properties[key]
#
#    ous_init.append(row)

KeyError: 'computers_names'

In [ ]:
df_groups=pd.DataFrame.from_dict(groups_init)
df_users=pd.DataFrame.from_dict(users_init)
df_computers=pd.DataFrame.from_dict(computers_init)
df_ous=pd.DataFrame.from_dict(ous_init)

In [ ]:
df_groups.columns

Index(['Members', 'Aces', 'ObjectIdentifier', 'IsDeleted', 'IsACLProtected',
       'ContainedBy', 'domain', 'name', 'distinguishedname', 'domainsid',
       'samaccountname', 'isaclprotected', 'description', 'whencreated',
       'admincount', 'reconcile'],
      dtype='object')

In [ ]:
df_groups.head(5)

,Members,Aces,ObjectIdentifier,IsDeleted,IsACLProtected,ContainedBy,domain,name,distinguishedname,domainsid,samaccountname,isaclprotected,description,whencreated,admincount,reconcile
0,[{'ObjectIdentifier': 'S-1-5-21-2697957641-227...,[{'PrincipalSID': 'S-1-5-21-2697957641-2271029...,PHANTOM.CORP-S-1-5-32-550,False,True,"{'ObjectIdentifier': 'S-1-5-32', 'ObjectType':...",PHANTOM.CORP,PRINT OPERATORS@PHANTOM.CORP,"CN=PRINT OPERATORS,CN=BUILTIN,DC=PHANTOM,DC=CORP",S-1-5-21-2697957641-2271029196-387917394,Print Operators,True,Members can administer printers installed on d...,1.653362e+09,True,NaN
1,[{'ObjectIdentifier': 'S-1-5-21-2697957641-227...,"[{'PrincipalSID': 'PHANTOM.CORP-S-1-5-32-544',...",PHANTOM.CORP-S-1-5-32-555,False,False,"{'ObjectIdentifier': 'S-1-5-32', 'ObjectType':...",PHANTOM.CORP,REMOTE DESKTOP USERS@PHANTOM.CORP,"CN=REMOTE DESKTOP USERS,CN=BUILTIN,DC=PHANTOM,...",S-1-5-21-2697957641-2271029196-387917394,Remote Desktop Users,False,Members in this group are granted the right to...,1.653362e+09,False,NaN
2,"[{'ObjectIdentifier': 'PHANTOM.CORP-S-1-5-17',...","[{'PrincipalSID': 'PHANTOM.CORP-S-1-5-32-544',...",PHANTOM.CORP-S-1-5-32-568,False,False,"{'ObjectIdentifier': 'S-1-5-32', 'ObjectType':...",PHANTOM.CORP,IIS_IUSRS@PHANTOM.CORP,"CN=IIS_IUSRS,CN=BUILTIN,DC=PHANTOM,DC=CORP",S-1-5-21-2697957641-2271029196-387917394,IIS_IUSRS,False,Built-in group used by Internet Information Se...,1.653362e+09,False,NaN
3,[{'ObjectIdentifier': 'S-1-5-21-2697957641-227...,[{'PrincipalSID': 'S-1-5-21-2697957641-2271029...,PHANTOM.CORP-S-1-5-32-548,False,True,"{'ObjectIdentifier': 'S-1-5-32', 'ObjectType':...",PHANTOM.CORP,ACCOUNT OPERATORS@PHANTOM.CORP,"CN=ACCOUNT OPERATORS,CN=BUILTIN,DC=PHANTOM,DC=...",S-1-5-21-2697957641-2271029196-387917394,Account Operators,True,Members can administer domain user and group a...,1.653362e+09,True,NaN
4,[],"[{'PrincipalSID': 'PHANTOM.CORP-S-1-5-32-544',...",PHANTOM.CORP-S-1-5-32-577,False,False,"{'ObjectIdentifier': 'S-1-5-32', 'ObjectType':...",PHANTOM.CORP,RDS MANAGEMENT SERVERS@PHANTOM.CORP,"CN=RDS MANAGEMENT SERVERS,CN=BUILTIN,DC=PHANTO...",S-1-5-21-2697957641-2271029196-387917394,RDS Management Servers,False,Servers in this group can perform routine admi...,1.653362e+09,False,NaN


In [ ]:
df_users

,AllowedToDelegate,PrimaryGroupSID,HasSIDHistory,SPNTargets,UnconstrainedDelegation,DomainSID,Aces,ObjectIdentifier,IsDeleted,IsACLProtected,...,userpassword,unixpassword,unicodepassword,sfupassword,logonscript,admincount,sidhistory,gmsa,msa,reconcile
0,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
1,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
2,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,False,[],NaN,NaN,NaN
3,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
4,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
5,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
6,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
7,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
8,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN
9,[],S-1-5-21-1333368235-828418111-1849936505-513,[],[],False,S-1-5-21-1333368235-828418111-1849936505,[{'PrincipalSID': 'S-1-5-21-1333368235-8284181...,S-1-5-21-1333368235-828418111-1849936505-500,False,True,...,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN


In [ ]:
df_groups.shape

(115, 16)

In [ ]:
df_users.shape

(53, 45)